In [280]:
import pandas as pd
import numpy as np
import openpyxl

In [244]:
df1 = pd.read_excel('./data/시세추이.xlsx')

In [246]:
# ATR(Average True Range) 변동성 지표 생성

df_atr = df1.iloc[:,[0,1,5,6]]
df_atr.columns = ['date', 'fin', 'high', 'low']

In [250]:
lst = []
atr = []

for i in range(1,2465):
    a = df_atr.iloc[i,2] - df_atr.iloc[i,3]
    b = df_atr.iloc[i,2] - df_atr.iloc[i-1,1]
    c = df_atr.iloc[i,3] - df_atr.iloc[i-1,1]
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))

del atr[820:823]

In [253]:
# y값(종가 데이터) 준비

fin = list(df_atr[1:-3]['fin'])
del fin[820:823]

In [ ]:
# 거래량 데이터 추출

df1 = df1[['일자', '거래량']]

In [ ]:
# PER, PBR 추출

df2 = pd.read_excel('./data/PERPBR.xlsx')
df2 = df2[['일자', 'PER', 'PBR']]

In [ ]:
# 거래주체별 순매수 데이터 추출

df3 = pd.read_excel('./data/거래실적.xlsx')
df3 = df3.drop(columns=['전체'], axis=1)

In [ ]:
# 데이터 병합
samsung = pd.merge(df1, df2, on='일자', how='outer')
samsung = pd.merge(samsung, df3, on='일자', how='outer')
samsung['ATR'] = atr

In [260]:
# '2018/04/27' 액면분할에 따른 전처리

samsung.iloc[820:,8] = samsung.iloc[820:,8]/50
samsung.iloc[820:,1] = samsung.iloc[820:,1]*50

In [ ]:
# 날짜 데이터로 변환

samsung['일자'] = pd.to_datetime(samsung['일자'])
samsung.rename(columns={'일자':'DATE'}, inplace=True)

In [299]:
# 거시경제 지표와 병합

macro_data = pd.read_csv('./data/merge_data.csv')
macro_data['DATE'] = pd.to_datetime(macro_data['DATE'])    # 날짜 데이터로 변환
macro_data = macro_data.dropna(axis=0)                     # 결측치가 포함된 행 제거
df_samsung = pd.merge(samsung, macro_data, how='left', on='DATE')

In [307]:
df_samsung

,DATE,거래량,PER,PBR,기관 합계,기타법인,개인,외국인 합계,ATR,NASDAQ,S&P,CBOE,Exchange rate,futures2y,futures10y,y
2460,2011-09-01,35870550,7.27,1.24,-83833734000,-8749641000,-177822918000,270406293000,880.0,"2,546.00","1,204.42",31.82,"1,064.40",110.28,130.61,15420.0
2459,2011-09-02,12934100,7.26,1.23,-7185878000,2125969000,-11377060000,16436969000,1000.0,"2,480.30","1,173.97",33.92,"1,065.60",110.25,131.30,15380.0
2458,2011-09-05,13874400,6.90,1.17,-975699000,-215641000,29370756000,-28179416000,580.0,"2,480.30","1,173.97",33.92,"1,065.60",110.25,131.30,14620.0
2457,2011-09-06,13225250,6.86,1.17,2349662000,-1299409000,13839817000,-14890070000,1260.0,"2,473.80","1,165.24",37.00,"1,074.65",110.27,131.22,14540.0
2456,2011-09-07,17575200,7.29,1.24,113232351000,1123519000,-77907268000,-36448602000,1300.0,"2,548.90","1,198.62",33.38,"1,069.65",110.27,130.89,15460.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021-08-25,22319664,19.71,1.92,-76990474100,-8894667500,82373232300,3511909300,2000.0,"15,041.90","4,496.19",16.79,"1,163.87",110.25,133.63,75700.0
3,2021-08-26,16671494,19.42,1.89,-55566537800,9344981600,142574757300,-96353201100,1900.0,"14,945.80","4,469.91",18.84,"1,170.53",110.25,133.59,74600.0
2,2021-08-27,15172748,19.34,1.89,173118394100,10372274300,1510887500,-185001555900,1200.0,"15,129.50","4,509.37",16.39,"1,161.23",110.30,133.89,74300.0
1,2021-08-30,12686999,19.42,1.89,24586418600,-2401179400,54715193800,-76900433000,2500.0,"15,265.90","4,528.79",16.19,"1,164.34",110.32,134.13,74600.0


In [301]:
# y값(종가 데이터) 병합

df_samsung['y'] = fin

In [302]:
# 종가 열 액면분할 반영

df_samsung.iloc[820:,15] = df_samsung.iloc[820:,15]/50

In [303]:
# 결측치를 전일 값으로 대체

df_samsung.sort_values('DATE', inplace=True)
df_samsung.fillna(method='ffill', inplace=True)

In [308]:
df_samsung.to_excel('./data/samsung.xlsx',index=False)

In [309]:
samsung_df = pd.read_excel('./data/samsung.xlsx')

In [311]:
samsung_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATE           2461 non-null   datetime64[ns]
 1   거래량            2461 non-null   int64         
 2   PER            2461 non-null   float64       
 3   PBR            2461 non-null   float64       
 4   기관 합계          2461 non-null   int64         
 5   기타법인           2461 non-null   int64         
 6   개인             2461 non-null   int64         
 7   외국인 합계         2461 non-null   int64         
 8   ATR            2461 non-null   int64         
 9   NASDAQ         2461 non-null   object        
 10  S&P            2461 non-null   object        
 11  CBOE           2461 non-null   float64       
 12  Exchange rate  2461 non-null   object        
 13  futures2y      2461 non-null   float64       
 14  futures10y     2461 non-null   float64       
 15  y              2461 n